## Fake NEWS Classifier Using Bidirectional LSTM

In [2]:
## Dataset: https://www.kaggle.com/competitions/fake-news/data?select=train.csv
import pandas as pd



In [3]:
df=pd.read_csv('/content/train_news.csv')

In [4]:
df.head()

,Unnamed: 0,id,headline,written_by,news,label
0,0,9653,Ethics Questions Dogged Agriculture Nominee as...,Eric Lipton and Steve Eder,"WASHINGTON — In Sonny Perdue’s telling, Geo...",0
1,1,10041,U.S. Must Dig Deep to Stop Argentina’s Lionel ...,David Waldstein,HOUSTON — Venezuela had a plan. It was a ta...,0
2,2,19113,Cotton to House: ’Do Not Walk the Plank and Vo...,Pam Key,"Sunday on ABC’s “This Week,” while discussing ...",0
3,3,6868,"Paul LePage, Besieged Maine Governor, Sends Co...",Jess Bidgood,"AUGUSTA, Me. — The beleaguered Republican g...",0
4,4,7596,A Digital 9/11 If Trump Wins,Finian Cunningham,Finian Cunningham has written extensively on...,1


In [5]:
df.isnull().sum()

,0
Unnamed: 0,0
id,0
headline,558
written_by,1957
news,39
label,0


In [6]:
df.shape

(20800, 6)

In [7]:
df=df.dropna()

In [8]:
df.head()

,Unnamed: 0,id,headline,written_by,news,label
0,0,9653,Ethics Questions Dogged Agriculture Nominee as...,Eric Lipton and Steve Eder,"WASHINGTON — In Sonny Perdue’s telling, Geo...",0
1,1,10041,U.S. Must Dig Deep to Stop Argentina’s Lionel ...,David Waldstein,HOUSTON — Venezuela had a plan. It was a ta...,0
2,2,19113,Cotton to House: ’Do Not Walk the Plank and Vo...,Pam Key,"Sunday on ABC’s “This Week,” while discussing ...",0
3,3,6868,"Paul LePage, Besieged Maine Governor, Sends Co...",Jess Bidgood,"AUGUSTA, Me. — The beleaguered Republican g...",0
4,4,7596,A Digital 9/11 If Trump Wins,Finian Cunningham,Finian Cunningham has written extensively on...,1


In [9]:
## get the independent features
X=df.drop('label',axis=1)

In [10]:
y=df['label']

In [11]:
## Check whether dataset is balanced or not
y.value_counts()

,count
label,
0,10361
1,7924


In [12]:
import tensorflow as tf

In [13]:
tf.__version__

'2.19.0'

In [14]:
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Bidirectional


In [15]:
## vocabulary size
voc_size=5000

In [17]:
messages=X.copy()

In [18]:
messages.head(10)

,Unnamed: 0,id,headline,written_by,news
0,0,9653,Ethics Questions Dogged Agriculture Nominee as...,Eric Lipton and Steve Eder,"WASHINGTON — In Sonny Perdue’s telling, Geo..."
1,1,10041,U.S. Must Dig Deep to Stop Argentina’s Lionel ...,David Waldstein,HOUSTON — Venezuela had a plan. It was a ta...
2,2,19113,Cotton to House: ’Do Not Walk the Plank and Vo...,Pam Key,"Sunday on ABC’s “This Week,” while discussing ..."
3,3,6868,"Paul LePage, Besieged Maine Governor, Sends Co...",Jess Bidgood,"AUGUSTA, Me. — The beleaguered Republican g..."
4,4,7596,A Digital 9/11 If Trump Wins,Finian Cunningham,Finian Cunningham has written extensively on...
6,6,5134,Rapid Evolution Saved This Fish From Pollution...,JoAnna Klein,The State of New Jersey says you can’t eat the...
7,7,1504,Alabama Prison Officials Retaliate Against Pri...,Brian Sonenstein,Advocates say prison officials at the Kilby Co...
9,9,4203,Can We Live in a Constant State of Love?,Gillian,Leave a reply \nToni Emerson – When we fall in...
10,10,11136,Is Hillary a Closet Drunk… New WikiLeak Email ...,V Saxena,BREAKING: Clinton Fixer Reveals Sick Smear Hil...
11,11,2150,BLOOMBERG-BACKED PENNSYLVANIA ATTORNEY GENERAL...,Iron Sheik,Home › POLITICS | US NEWS › BLOOMBERG-BACKED P...


In [19]:
import nltk
import re
from nltk.corpus import stopwords

In [20]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [21]:
messages.reset_index(inplace=True)

In [24]:
### Dataset Preprocessing
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()
corpus = []
for i in range(0, len(messages)):
    print(i)
    review = re.sub('[^a-zA-Z]', ' ', messages['headline'][i])
    review = review.lower()
    review = review.split()

    review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
    review = ' '.join(review)
    corpus.append(review)

Streaming output truncated to the last 5000 lines.
13285
13286
13287
13288
13289
13290
13291
13292
13293
13294
13295
13296
13297
13298
13299
13300
13301
13302
13303
13304
13305
13306
13307
13308
13309
13310
13311
13312
13313
13314
13315
13316
13317
13318
13319
13320
13321
13322
13323
13324
13325
13326
13327
13328
13329
13330
13331
13332
13333
13334
13335
13336
13337
13338
13339
13340
13341
13342
13343
13344
13345
13346
13347
13348
13349
13350
13351
13352
13353
13354
13355
13356
13357
13358
13359
13360
13361
13362
13363
13364
13365
13366
13367
13368
13369
13370
13371
13372
13373
13374
13375
13376
13377
13378
13379
13380
13381
13382
13383
13384
13385
13386
13387
13388
13389
13390
13391
13392
13393
13394
13395
13396
13397
13398
13399
13400
13401
13402
13403
13404
13405
13406
13407
13408
13409
13410
13411
13412
13413
13414
13415
13416
13417
13418
13419
13420
13421
13422
13423
13424
13425
13426
13427
13428
13429
13430
13431
13432
13433
13434
13435
13436
13437
13438
13439
13440
13441
13442
1

In [25]:
corpus

['ethic question dog agricultur nomine georgia governor new york time',
 'u must dig deep stop argentina lionel messi new york time',
 'cotton hous walk plank vote bill cannot pass senat breitbart',
 'paul lepag besieg main governor send conflict signal intent new york time',
 'digit trump win',
 'rapid evolut save fish pollut studi say new york time',
 'alabama prison offici retali prison strike leader cut water cell',
 'live constant state love',
 'hillari closet drunk new wikileak email reveal aid ask sober day',
 'bloomberg back pennsylvania attorney gener sentenc month prison',
 'video le pen pelt egg keep smile',
 'donald trump candid american special interest',
 'wharton busi school backer seek distanc donald trump new york time',
 'brexit rule latest tactic block peopl',
 'million south korean rise shadow govern',
 'chelsea man ask obama cut sentenc time serv new york time',
 'citi vow fight trump immigr even lose million new york time',
 'youtub livestream pregnant giraff pull

In [26]:
onehot_repr=[one_hot(words,voc_size) for words in corpus]
onehot_repr

[[4769, 3473, 4150, 4979, 2945, 2777, 2412, 3628, 4325, 322],
 [3416, 1320, 2648, 3071, 1508, 3236, 257, 3972, 3628, 4325, 322],
 [3153, 1894, 4907, 4320, 2293, 2038, 1655, 1098, 2592, 3808],
 [4584, 3657, 1961, 775, 2412, 1829, 4399, 2192, 1088, 3628, 4325, 322],
 [153, 799, 1086],
 [1075, 2946, 2225, 4548, 1063, 3090, 2984, 3628, 4325, 322],
 [268, 927, 3444, 2268, 927, 2454, 2773, 1694, 2425, 2387],
 [3231, 4678, 1244, 1544],
 [75, 3295, 4304, 3628, 3498, 418, 1075, 1935, 2738, 2380, 1071],
 [521, 1754, 2476, 2166, 2159, 3591, 4683, 927],
 [3462, 3820, 3426, 2666, 3472, 1180, 4802],
 [1329, 799, 2739, 1723, 3896, 1169],
 [2296, 2569, 930, 1016, 2593, 4305, 1329, 799, 3628, 4325, 322],
 [3206, 4418, 4781, 2143, 4475, 4168],
 [1676, 3989, 3982, 2449, 2513, 2310],
 [3392, 289, 2738, 4372, 1694, 3591, 322, 70, 3628, 4325, 322],
 [3645, 1572, 4639, 799, 2029, 3139, 335, 1676, 3628, 4325, 322],
 [3597, 4108, 3468, 3009, 2950, 3668, 83, 952, 3808],
 [3930, 2577, 3628, 4923, 1630, 3628, 432

## Embedding Representation

In [27]:
sent_length=20
embedded_docs=pad_sequences(onehot_repr,padding='pre',maxlen=sent_length)

embedded_docs


array([[   0,    0,    0, ..., 3628, 4325,  322],
       [   0,    0,    0, ..., 3628, 4325,  322],
       [   0,    0,    0, ..., 1098, 2592, 3808],
       ...,
       [   0,    0,    0, ..., 2015, 1890, 3474],
       [   0,    0,    0, ..., 3628, 4325,  322],
       [   0,    0,    0, ..., 3001, 1935, 3808]], dtype=int32)

In [35]:
## Creating model
embedding_vector_features= 40
model=Sequential()
model.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
model.add(Bidirectional(LSTM(100)))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model.summary())

Model: "sequential_7"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_7 (Embedding)         │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional_7 (Bidirectional) │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_7 (Dense)                 │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

None


In [36]:
import numpy as np
X_final=np.array(embedded_docs)
y_final=np.array(y)

In [37]:
## train test split

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_final, y_final, test_size=0.33, random_state=42)

In [38]:
## Model Training
model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=10,batch_size=32)

Epoch 1/10
383/383 ━━━━━━━━━━━━━━━━━━━━ 12s 12ms/step - accuracy: 0.8110 - loss: 0.3727 - val_accuracy: 0.9188 - val_loss: 0.1894
Epoch 2/10
383/383 ━━━━━━━━━━━━━━━━━━━━ 4s 10ms/step - accuracy: 0.9439 - loss: 0.1381 - val_accuracy: 0.9239 - val_loss: 0.1911
Epoch 3/10
383/383 ━━━━━━━━━━━━━━━━━━━━ 5s 12ms/step - accuracy: 0.9661 - loss: 0.0902 - val_accuracy: 0.9238 - val_loss: 0.2014
Epoch 4/10
383/383 ━━━━━━━━━━━━━━━━━━━━ 4s 10ms/step - accuracy: 0.9814 - loss: 0.0602 - val_accuracy: 0.9215 - val_loss: 0.2577
Epoch 5/10
383/383 ━━━━━━━━━━━━━━━━━━━━ 5s 14ms/step - accuracy: 0.9882 - loss: 0.0332 - val_accuracy: 0.9210 - val_loss: 0.3056
Epoch 6/10
383/383 ━━━━━━━━━━━━━━━━━━━━ 4s 11ms/step - accuracy: 0.9941 - loss: 0.0198 - val_accuracy: 0.9142 - val_loss: 0.4104
Epoch 7/10
383/383 ━━━━━━━━━━━━━━━━━━━━ 4s 9ms/step - accuracy: 0.9958 - loss: 0.0123 - val_accuracy: 0.9190 - val_loss: 0.3736
Epoch 8/10
383/383 ━━━━━━━━━━━━━━━━━━━━ 4s 10ms/step - accuracy: 0.9978 - loss: 0.0085 - val_accu

In [39]:
y_pred=model.predict(X_test)

189/189 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step


In [40]:
import numpy as np

y_pred=np.where(y_pred>=0.5,1,0)

In [41]:
from sklearn.metrics import confusion_matrix,accuracy_score
confusion_matrix(y_test,y_pred)

array([[3159,  254],
       [ 240, 2382]])

In [42]:
print(accuracy_score(y_test,y_pred))

0.9181441590720796
